<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd #импортируем библиотеки
from nltk.stem.wordnet import WordNetLemmatizer
import re 
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import torch
import transformers
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

Загрузим и подготовим данные:

In [2]:
data =  pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
data.info() # информация о DF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные чистые, без пропусков. Целевой признак - "toxic". 

In [4]:
data.head(5) # посмотрим как выглядят

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Введем функцию лемматизации и очистки данных:

In [5]:
nltk.download('stopwords') # загрузим стоп-слова
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clear_text(text):
    text_clear = ' '.join(re.sub(r'[^a-zA-z]', ' ', text).lower().split())
    word_list = nltk.word_tokenize(text_clear)
    return word_list

In [7]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    lemm_text = [lemmatizer.lemmatize(w) for w in text]
    return " ".join(lemm_text)

Применим функцию к данным:

In [8]:
data["text"] = data["text"].apply(clear_text)  

In [9]:
data["text"] = data["text"].apply(lemmatize)  

In [28]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestion on impro...,0
4,you sir are my hero any chance you remember wh...,0


Выделим в данных целевой признак:

In [11]:
target = data['toxic']
features = data.drop(['toxic'], axis=1)

Разделим данные на выборки (обучающая и тестовая): 

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=42)

Загрузим пакет стоп-слов, это слова, которые не несут смысловой нагрузки. 

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) # создадим счетчик, указав в нем стоп-слова

Посчитаем TF-IDF для текста, применив fit_transform. Функцию fit запускаем только на обучающей выборке, иначе тестирование будет нечестным, в модели будут учтены частоты слов из тестовой выборки. На тестовой выборке только transform. 

In [14]:
features_train = count_tf_idf.fit_transform(features_train["text"].values)

In [15]:
features_test = count_tf_idf.transform(features_test['text'].values) 

## Обучение

Рассмотрим на модели Логистическая регрессия:

In [ ]:
model_lr = LogisticRegression(random_state = 42, class_weight='balanced')
params = {
   #'penanlty': ['l1', 'l2'],        
   'C': [0.5, 1.0, 5.0, 15.0],
    'intercept_scaling': range(5, 100, 40), 
    'solver': ['liblinear'] 
}
lr_gs = GridSearchCV(model_lr, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)
lr_gs.best_params_

In [17]:
model_lr_best_param = LogisticRegression(random_state = 42, C = 5, intercept_scaling = 5, solver = 'liblinear')

In [27]:
model_lr_best_param.fit(features_train, target_train)
predictions_lr = model_lr_best_param.predict(features_test)
f1_lr = f1_score(target_test, predictions_lr)
print("Для модели Логистическая регрессия f1:", f1_lr)

Для модели Логистическая регрессия f1: 0.774861


Рассмотрим на модели Дерево решений:

In [19]:
tree = DecisionTreeClassifier(random_state = 42, class_weight='balanced')
params = {
   'criterion':['gini', 'entropy'],        
   'max_depth':list(range(1,15,5)) 
}
tree_gs = GridSearchCV(tree, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)
tree_gs.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


{'criterion': 'entropy', 'max_depth': 11}

In [29]:
tree_best = DecisionTreeClassifier(criterion = 'entropy', max_depth = 11, random_state = 42)

In [32]:
tree_best.fit(features_train, target_train) # обучим
predictions_tree = tree_best.predict(features_test)  # предскажем
f1_tree = f1_score(target_test, predictions_tree)
print("Для модели Дерево решений метрика f1:", f1_tree)

Для модели Дерево решений метрика f1: 0.5875090029838461


Рассмотрим на модели Случайный лес:

In [33]:
model_forest = RandomForestClassifier(random_state=42, class_weight='balanced')

In [34]:
param_grid = {'n_estimators': [1, 100], 'max_depth': [5, 10, 12]}
gs = GridSearchCV(model_forest, param_grid, cv=3, scoring='f1')
gs.fit(features_train, target_train)
gs.best_params_ 

{'max_depth': 12, 'n_estimators': 100}

In [35]:
model_forest_optim = RandomForestClassifier(random_state = 42, max_depth = 12, n_estimators = 100)
model_forest_optim.fit(features_train, target_train)
predictions_forest = model_forest_optim.predict(features_test)

In [36]:
f1_forest = f1_score(target_test, predictions_forest)
print("Для модели Случайный лес f1:", f1_forest)

Для модели Случайный лес f1: 0.0


## Выводы

С полученными данными провели очистку и лемматизацию, выдели целевой признак, разбили на выбрки. Посчитали TF-IDF для текста, загрузив стоп-слова. Обучили данные на примере моделей Логистической регрессии, Дерева решений, Случайного леса классифицировтаь комментарии на позитивные и негативные. Для всех моделей определили метрику f1. Данные приведены в таблице. Лучший результат показала модель Логистическая регрессия. 

In [37]:
rez = pd.DataFrame({"модель": ["Логистическая регрессия", "Дерево решений", 
                              "Случайный лес"], "значение метриик f1": [f1_lr, f1_tree, f1_forest]})
rez

,модель,значение метриик f1
0,Логистическая регрессия,0.774861
1,Дерево решений,0.587509
2,Случайный лес,0.000000


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны